In [1]:
import sys
import os
import dedupe
from __future__ import print_function

Sometimes you want to know what functions are being called - for example when you're trying to figure out how a complex package works. Fortunately, Python makes this pretty easy - the `sys` package contains a `settrace` method that allows you to do just that. Unfortunately, it's a bit eager, and doesn't work particularly well in a notebook environment without a bit of tweaking. That's because `settrace` will tell you _all_ the functions that are called - even those that are just to do with executing code from the notebook itself.

So... I wrote a function that seems to handle all that faff pretty well. Here it is:

In [2]:
def tracefunc(frame, event, arg, package_name='dedupe'):
    """ Function to track the methods, classes and filenames that are called from Jupyter notebook.
        Will need some tweaking for use outside of notebook environment.
        Useful for tracing what functions have been used in complex packages.
        Inspired by: http://stackoverflow.com/questions/8315389/how-do-i-print-functions-as-they-are-called
        
        Inputs: - frame, event, arg -> See the docs: https://docs.python.org/3/library/sys.html#sys.settrace
                - package_name -> string of the package you want to track
                                  (without this get a load of notebook faff printed out too)
        
        Outputs: - prints to screen the filename, class and methods called by running cell.
        
        Usage: Turn on -> sys.settrace(tracefunc)
               Turn off -> sys.settrace(None)
    """
    
    file_name = frame.f_code.co_filename
    func_name = frame.f_code.co_name
    
    # this is a bit of a hack to get the class out of the locals
    # - it relies on 'self' being used... normally a safe assumption!
    try:
        class_name = frame.f_locals['self'].__class__.__name__ 
    except (KeyError, AttributeError):
        class_name = "No Class"
                  
    # notebook filenames appear as 'ipython-input...'
    if 'ipython-input' in file_name or package_name in file_name:
        # another thing that appears to be specific to notebooks
        if '<module>' not in func_name:
            # this is where the work gets done!
            if event == "call":
                print("Filename: " + os.path.basename(file_name) + \
                      " -> Class: " + class_name + \
                      " -> Function: " + func_name)
    return tracefunc

To show `settrace` in action, here's a toy class I created:

In [3]:
class foo:
    def __init__(self):
        self.x = 4
        print("got here")
    
    def bar(self):
        self.y = 5

You use it as follows - first turn it on via:

In [4]:
sys.settrace(tracefunc)

I now call the foo class (and its bar method) from a notebook cell:

In [5]:
f = foo()
f.bar()

Filename: <ipython-input-3-72f85d199f0d> -> Class: foo -> Function: __init__
got here
Filename: <ipython-input-3-72f85d199f0d> -> Class: foo -> Function: bar


... and you get a print out of what is being called.

For something more complex I'm using `dedupe`, which is a cracking package for entity resolution in Python. First you define the fields which you care about, then you feed that into the `Dedupe` class, which runs off and does a load of initialisations - `settrace` follows these:

In [6]:
fields = [{'field' : 'family_name', 'type' : 'String'},
          {'field' : 'given_name', 'type': 'String'}, 
          {'field' : 'middle_name', 'type': 'String', 'has_missing' : True},
          {'field' : 'dob_year', 'type': 'String', 'has_missing' : True},
          {'field' : 'dob_month', 'type': 'String', 'has_missing' : True}]

In [7]:
deduper = dedupe.Dedupe(fields)

Filename: api.py -> Class: Dedupe -> Function: __init__
Filename: api.py -> Class: Dedupe -> Function: __init__
Filename: datamodel.py -> Class: DataModel -> Function: __init__
Filename: datamodel.py -> Class: No Class -> Function: typifyFields
Filename: string.py -> Class: StringType -> Function: __init__
Filename: string.py -> Class: StringType -> Function: __init__
Filename: predicates.py -> Class: StringPredicate -> Function: __init__
Filename: predicates.py -> Class: StringPredicate -> Function: __init__
Filename: predicates.py -> Class: StringPredicate -> Function: __init__
Filename: predicates.py -> Class: StringPredicate -> Function: __init__
Filename: predicates.py -> Class: StringPredicate -> Function: __init__
Filename: predicates.py -> Class: StringPredicate -> Function: __init__
Filename: predicates.py -> Class: StringPredicate -> Function: __init__
Filename: predicates.py -> Class: StringPredicate -> Function: __init__
Filename: predicates.py -> Class: StringPredicate -> 

Et voila, that's about it. To turn off the tracing, just run:

In [8]:
sys.settrace(None)